In [ ]:
import datetime
import os

import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('nbs.mplstyle')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd

os.makedirs('../plots/', exist_ok=True)

In [ ]:
astropy_df = pd.read_csv('../cache/astropy-citations.csv',
                         index_col='pubdate', parse_dates=True)

In [ ]:
N = len(astropy_df['authors'].str.split(';').explode().unique())
print(f"Total number of unique authors that cite Astropy: {N}")

# Journal statistics:

In [ ]:
unq_journals, counts = np.unique(astropy_df.pub.values.astype(str), 
                                 return_counts=True)

In [ ]:
print(f"Number of journals that cite astropy: {len(unq_journals)}")

In [ ]:
astropy_df['pub'].astype(str).value_counts()[:20]

## Top 10 authors that cite astropy as lead author:

In [ ]:
astropy_df['authors'].str.split(';').apply(lambda x: x[0]).value_counts()[:10]

## Top 25 authors that cite Astropy on co-authored papers:

In [ ]:
astropy_df['authors'].str.split(';').explode().value_counts()[:20]

## Number of citations per month:

In [ ]:
g = astropy_df.groupby(by=[astropy_df.index.year, astropy_df.index.month]).count()

group_dates = [datetime.date(x[0], x[1], 1) for x in g.index]
group_dates_dec = [x[0] + x[1]/12 for x in g.index]

for add_lines in [True, False]:
    fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=True)
    ax.plot(group_dates, g['pubdate.1'].values, 
            marker='', drawstyle='steps-mid', 
            lw=2, color='tab:orange')
    ax.set_xlim(datetime.date(2013, 8, 1),
                datetime.datetime.now())

    ax.set_xlabel('time')
    ax.set_ylabel('Astropy papers\nrefereed citations per month')
    
    if add_lines:
        ylim = ax.get_ylim()
        yptp = ylim[1] - ylim[0]
        
        ax.axvline(datetime.date(2013, 10, 1), 
                   zorder=-1, linestyle='--')
        ax.axvline(datetime.date(2018, 9, 1), 
                   zorder=-1, linestyle='--')
        
        ax.text(datetime.date(2018, 10-2, 1), 
                ylim[1] - 0.05 * yptp,
                'Astropy paper 2',
                ha='right', va='top', fontsize=16)
        
        ax.text(datetime.date(2013, 10+1, 1), 
                ylim[1] - 0.05 * yptp,
                'Astropy paper 1',
                ha='left', va='top', fontsize=16)
    
    extra = '-lines' if add_lines else ''
    
    fig.text(1, 0, 'Source: NASA/ADS', fontsize=16, ha='right')
    
    fig.savefig(f'../plots/astropy-citations{extra}.png', dpi=300)

In [ ]:
all_pubs = np.unique(astropy_df['pub'].values.astype(str))

In [ ]:
g = astropy_df.groupby(by=[astropy_df.index.year, 
                           astropy_df.pub]).count()

group_dates = np.array([datetime.date(x[0], 1, 1) for x in g.index])
group_dates_dec = [x[0] for x in g.index]
pubs = [str(x[1]) for x in g.index]

# fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=True)
# ax.plot(group_dates, g['pubdate.1'].values, 
#         marker='', drawstyle='steps-mid', 
#         lw=2, color='tab:orange')
# ax.set_xlim(datetime.date(2013, 8, 1),
#             datetime.datetime.now())

# ax.set_xlabel('time')
# ax.set_ylabel('Astropy papers\nrefereed citations per month')

# fig.text(1, 0, 'Source: NASA/ADS', fontsize=16, ha='right')

# # fig.savefig(f'../plots/astropy-citations{extra}.png', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=True)

for pub in all_pubs:
    mask = pubs == np.array(pub)
    
    gg = g[mask]
    dates = group_dates[mask]
    if gg['year'].values.sum() < 50:
        continue
        
    ax.plot(dates, gg['year'].values, 
            marker='', #drawstyle='steps-mid', 
            lw=2, label=pub)   
    
ax.set_xlim(datetime.date(2013, 8, 1),
            datetime.datetime.now())

ax.set_xlabel('time')
ax.set_ylabel('Astropy papers\nrefereed citations per year')

ax.set_yscale('log')

ax.legend()

fig.text(1, 0, 'Source: NASA/ADS', fontsize=16, ha='right')

fig.savefig(f'../plots/astropy-citations-per-journal.png', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=True)

for pub in all_pubs:
    mask = pubs == np.array(pub)
    
    gg = g[mask]
    dates = group_dates[mask]
    if gg['year'].values.sum() >= 50 or gg['year'].values.sum() < 10:
        continue
        
    ax.plot(dates, gg['year'].values, 
            marker='', #drawstyle='steps-mid', 
            lw=2, label=pub)   
    
ax.set_xlim(datetime.date(2013, 8, 1),
            datetime.datetime.now())

ax.set_xlabel('time')
ax.set_ylabel('Astropy papers\nrefereed citations per year')

ax.set_yscale('log')

ax.legend()

fig.text(1, 0, 'Source: NASA/ADS', fontsize=16, ha='right')

fig.savefig(f'../plots/astropy-citations-per-journal-few.png', dpi=300)

Journals without many citations to Astropy:

In [ ]:
for pub in all_pubs:
    mask = pubs == np.array(pub)
    
    gg = g[mask]
    dates = group_dates[mask]
    if gg['year'].values.sum() >= 10:
        continue
        
    print(pub)